# Création d'un corpus d'entraînement pour Layout Parser à partir de données OLR de documents Gallica

Ce script permet de créer un corpus d'entraînement pour l'outil **ModOAP - Détection de mise en page (Entrainement)**, à partir d'annotations de mise en page (OLR) opérées sur des documents de presse numérisés en mode article et disponibles sur Gallica : [voir les corpus disponibles](https://api.bnf.fr/fr/documents-de-presse-numerises-en-mode-article) 

Le script nécessite de synchroniser un compte Google Drive sur lequel se trouve un dossier contenant les informations de mise en page téléchargées depuis https://api.bnf.fr/fr/documents-de-presse-numerises-en-mode-article (fichiers .xml contenus dans le dossier ocr)

Ce script utilise le protocole de récupération d'images IIIF de Gallica : https://api.bnf.fr/fr/api-iiif-de-recuperation-des-images-de-gallica

In [ ]:
#@title  Imports et synchronisation du drive
from google.colab import drive
import os

# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')
  
import glob
import gzip
import shutil
import json
from tqdm import tqdm
import urllib
import cv2
try : 
  import xmltodict
except ModuleNotFoundError : 
    
  !pip install xmltodict
  import xmltodict




def cb_traitement(cb, ratio_height, ratio_width, image_id, ark) :
  if isinstance(cb["TextBlock"],dict) :
    bloc_legende = {'@HPOS' : cb["TextBlock"]["@HPOS"], 
                    '@VPOS' : cb["TextBlock"]["@VPOS"], 
                    '@HEIGHT' : cb["TextBlock"]["@HEIGHT"], 
                    '@WIDTH' : cb["TextBlock"]["@WIDTH"], 
                    '@ID' : cb["TextBlock"]["@ID"], 
                    '@LANG' : cb["TextBlock"]["@LANG"]}

    couples_ill_txt[ark+"_"+cb["@ID"]] = {"bloc_illustration" : cb["Illustration"],
                                "bloc_legende" : bloc_legende ,
                                "ratio_height" : ratio_height,
                                "ratio_width" : ratio_width,
                                "image_id" : image_id}
  elif isinstance(cb["TextBlock"],list) :
    blocs_legende = []
    for tb in cb["TextBlock"] :
      bloc_legende = {'@HPOS' : tb["@HPOS"], 
                    '@VPOS' : tb["@VPOS"], 
                    '@HEIGHT' : tb["@HEIGHT"], 
                    '@WIDTH' : tb["@WIDTH"], 
                    '@ID' : tb["@ID"], 
                    '@LANG' : tb["@LANG"]}
      blocs_legende.append(bloc_legende)

      couples_ill_txt[ark+"_"+cb["@ID"]] = {"bloc_illustration" : cb["Illustration"],
                                    "bloc_legende" : blocs_legende,
                                    "ratio_height" : ratio_height,
                                    "ratio_width" : ratio_width,
                                    "image_id" : image_id}


**1. Préparer un dossier_corpus** contenant un sous-dossier par numéro de périodique contenant un fichier xml par page du document.

Exemple : 

-dossier_corpus

---465885
*  X00000001.xml
*  X00000002.xml
*  X00000003.xml

---475455
*  X00000001.xml
*  X00000002.xml
*  X00000003.xml
*  X00000004.xml

In [ ]:
#@markdown  #### 2. Transformer le dossier_corpus en corpus d'entraînement pour Layout Parser

dossier_corpus = "" #@param {type:"string"}

#@markdown  ##### Télécharge les pages du corpus au format jpg via IIF Gallica

#@markdown  ##### Récupère les annotations des fichiers OCR pour créer un fichier au format COCO Json

#@markdown  ##### Le dossier_corpus comprend au final deux sous-dossiers train et val, contenant chacun des pages du corpus au format jpg et un fichier d'annotation au format json

fichiers_total = []
print("Téléchargement des pages des documents et renommage des fichiers xml")

print()

for dossier in tqdm(glob.glob(os.path.join(dossier_corpus,"*"))) : 
  files = glob.glob(os.path.join(dossier,"*.xml"))
  with open(files[0],"r") as altin :
    alto = altin.read()
  altodic = xmltodict.parse(alto)
  ark2 = altodic['alto']["Description"]["sourceImageInformation"]["fileIdentifier"].split("/")[-2]
  for f in range(len(files)) :
    url = "https://gallica.bnf.fr/iiif/ark:/12148/{}/f{}/full/pct:20/0/native.jpg".format(ark2,f+1)
    nomfichier = ark2+"_"+"view_"+str(f)+".jpg"
    cheminout = os.path.join(dossier,nomfichier)
    try :
      urllib.request.urlretrieve(url, cheminout)
    except (HTTPError, URLError) as erreur:
      print(str(erreur.reason)) 

  fichiersjpg = [fi for fi in glob.glob(os.path.join(dossier,"*.jpg"))]
  fichiersxml = [fi for fi in glob.glob(os.path.join(dossier,"*.xml"))]
  fichiersxml.sort(key=lambda fname: int(fname.split('/')[-1].split(".")[0][1:]))
  fichiersjpg.sort(key=lambda fname: int(fname.split('_')[-1].split(".")[0]))
  for e in fichiersxml :
    fil_jpg = fichiersjpg[fichiersxml.index(e)].split("/")[-1][:-4]+".xml"
    chemin = "/".join(e.split("/")[:-1])
    os.rename(e,os.path.join(chemin,fil_jpg))
  
  fichiers_total = fichiers_total + [fi for fi in glob.glob(os.path.join(dossier,"*.xml"))] + fichiersjpg

  print(len(fichiers_total))

print("Séparation Test/Val")
fichiers_total.sort()
index_split= int(len(fichiers_total) * 80/100) 
if (index_split % 2) != 0:  
   index_split += 1
train = fichiers_total[:index_split]
val = fichiers_total[index_split:]
print("Train : {} fichiers".format(len(train)))

if not os.path.exists(os.path.join(dossier_corpus,"test")) :
  os.makedirs(os.path.join(dossier_corpus,"test"))
if not os.path.exists(os.path.join(dossier_corpus,"val")) :
  os.makedirs(os.path.join(dossier_corpus,"val"))
for fichier in tqdm(train) :
  shutil.move(fichier, os.path.join(dossier_corpus,"test"))

print()
print("Val : {} fichiers".format(len(val)))

for fichier in tqdm(val) :
  shutil.move(fichier, os.path.join(dossier_corpus,"val"))

for dossier in glob.glob(os.path.join(dossier_corpus,"*")):
  if not "val" in dossier and not "test" in dossier : 
    shutil.rmtree(dossier)

#############################################################################################################################
# Récupération des annotations

dossiers_images = [f for f in glob.glob(os.path.join(dossier_corpus,"*"))]

for dossier_images in dossiers_images :
  print("dossier : ", dossier_images)

  ######### Création de liste_images #############

  ######### Création de couples_ill_txt #############
  liste_images = []
  couples_ill_txt = {}

  fichiers_pages_jpg = [p for p in glob.glob(os.path.join(dossier_images,"*.jpg"))]
  #fichiers_pages_jpg.sort(key=lambda fname: int(fname.split('_')[-1].split(".")[0]))

  print("fichiers : ", fichiers_pages_jpg)

  image_id = 0

  for f in tqdm(fichiers_pages_jpg) :
      ark = f.split("/")[-1].split("_")[0]
      
      img = cv2.imread(f)
      height = img.shape[0]
      width = img.shape[1]
      file_name = f.split("/")[-1]
      image_id += 1
      dico_image = {"file_name" : file_name,
                    "height" : height,
                    "width" : width,
                    "id" : image_id}
      liste_images.append(dico_image)


      with open(f.replace(".jpg",".xml"), "r") as alt :
        alto = alt.read()
        altodic = xmltodict.parse(alto)
        height_ocr = int(altodic["alto"]["Layout"]["Page"]["@HEIGHT"])
        width_ocr = int(altodic["alto"]["Layout"]["Page"]["@WIDTH"])
        
        ratio_height = height / height_ocr
        ratio_width = width / width_ocr

        try :
          if isinstance(altodic['alto']['Layout']['Page']['PrintSpace']['ComposedBlock'], dict) :
            cb = altodic['alto']['Layout']['Page']['PrintSpace']['ComposedBlock']
            cb_traitement(cb, ratio_height, ratio_width, image_id, ark)
          elif isinstance(altodic['alto']['Layout']['Page']['PrintSpace']['ComposedBlock'], list) :
            for cb in altodic['alto']['Layout']['Page']['PrintSpace']['ComposedBlock'] :
              cb_traitement(cb, ratio_height, ratio_width, image_id, ark)
          else : print("PROBLEME")
        except KeyError : pass


  #  Extraction des annotations et enregistrement au format COCO

  chemin_sauvegarde = os.path.join(dossier_images,"annotations_COCO.json")
  print("sauvegarde dans : ", chemin_sauvegarde)

  ######### Création de anno #############

  anno = []
  n = 1
  for couple in tqdm(couples_ill_txt) :

    image_id = couples_ill_txt[couple]["image_id"]

    if isinstance(couples_ill_txt[couple]["bloc_illustration"], dict) :

      hposI = int(int(couples_ill_txt[couple]["bloc_illustration"]["@HPOS"]) * couples_ill_txt[couple]["ratio_width"])
      vposI = int(int(couples_ill_txt[couple]["bloc_illustration"]["@VPOS"]) * couples_ill_txt[couple]["ratio_height"])
      heightI = int(int(couples_ill_txt[couple]["bloc_illustration"]["@HEIGHT"]) * couples_ill_txt[couple]["ratio_height"])
      widthI = int(int(couples_ill_txt[couple]["bloc_illustration"]["@WIDTH"]) * couples_ill_txt[couple]["ratio_width"])

      segmentation = [hposI, vposI,hposI + widthI,vposI,hposI + widthI,vposI+heightI, hposI,vposI+heightI]
      bbox = [hposI, vposI, widthI ,heightI]
      area = heightI * widthI
      
      dico_anno = {"segmentation" : [segmentation], "area" : area , "iscrowd" : 0, "image_id" : image_id, "bbox" : bbox, "category_id" : 1, "id": n}
      n+= 1
      anno.append(dico_anno)

    elif isinstance(couples_ill_txt[couple]["bloc_illustration"], list) :
      for illus in couples_ill_txt[couple]["bloc_illustration"] :
        hposI = int(int(illus["@HPOS"]) * couples_ill_txt[couple]["ratio_width"])
        vposI = int(int(illus["@VPOS"]) * couples_ill_txt[couple]["ratio_height"])
        heightI = int(int(illus["@HEIGHT"]) * couples_ill_txt[couple]["ratio_height"])
        widthI = int(int(illus["@WIDTH"]) * couples_ill_txt[couple]["ratio_width"])

        segmentation = [hposI, vposI,hposI + widthI,vposI,hposI + widthI,vposI+heightI, hposI,vposI+heightI]
        bbox = [hposI, vposI,widthI,heightI]
        area = heightI * widthI
        
        dico_anno = {"segmentation" : [segmentation], "area" : area , "iscrowd" : 0, "image_id" : image_id, "bbox" : bbox, "category_id" : 1, "id": n}
        n+= 1
        anno.append(dico_anno)

    if isinstance(couples_ill_txt[couple]["bloc_legende"], dict) :

      hposL = int(int(couples_ill_txt[couple]["bloc_legende"]["@HPOS"]) * couples_ill_txt[couple]["ratio_width"])
      vposL = int(int(couples_ill_txt[couple]["bloc_legende"]["@VPOS"]) * couples_ill_txt[couple]["ratio_height"])
      heightL = int(int(couples_ill_txt[couple]["bloc_legende"]["@HEIGHT"]) * couples_ill_txt[couple]["ratio_height"])
      widthL = int(int(couples_ill_txt[couple]["bloc_legende"]["@WIDTH"]) * couples_ill_txt[couple]["ratio_width"])

      segmentation = [hposL, vposL,hposL + widthL,vposL,hposL + widthL,vposL+heightL, hposL,vposL+heightL]
      bbox = [hposL, vposL, widthL, heightL ]
      area = heightL * widthL

      dico_anno = {"segmentation" : [segmentation], "area" : area , "iscrowd" : 0, "image_id" : image_id, "bbox" : bbox, "category_id" : 2, "id": n}
      n+= 1
      anno.append(dico_anno)


    elif isinstance(couples_ill_txt[couple]["bloc_legende"], list) :
      for tb in couples_ill_txt[couple]["bloc_legende"] :


        hposL = int(int(tb["@HPOS"]) * couples_ill_txt[couple]["ratio_width"])
        vposL = int(int(tb["@VPOS"]) * couples_ill_txt[couple]["ratio_height"])
        heightL = int(int(tb["@HEIGHT"]) * couples_ill_txt[couple]["ratio_height"])
        widthL = int(int(tb["@WIDTH"]) * couples_ill_txt[couple]["ratio_width"])

        segmentation = [hposL, vposL,hposL + widthL,vposL,hposL + widthL,vposL+heightL, hposL,vposL+heightL]
        bbox = [hposL, vposL, widthL,heightL]
        area = heightL * widthL

        dico_anno = {"segmentation" : [segmentation], "area" : area , "iscrowd" : 0, "image_id" : image_id, "bbox" : bbox, "category_id" : 2, "id": n}
        n+= 1
        anno.append(dico_anno)
  ######### Création du Json final #############

  info = {}
  licenses = {}
  images = liste_images
  categories = [{'supercategory': 'illustration', 'id': 1, 'name': 'illustration'}, {'supercategory': 'legende', 'id': 2, 'name': 'legende'} ]

  cocofinal = {"info" : info,
              "licences" : licenses,
              "images" : images,
              "categories" : categories, 
              "annotations" : anno} 

  with open(chemin_sauvegarde, "w") as jout:
    json.dump(cocofinal, jout )

  for xml in [f for f in glob.glob(os.path.join(dossier_images,"*.xml"))]:
    os.remove(xml)
print()
print("Le dossier {} est prêt pour l'entraînement".format(dossier_corpus))


